<a href="https://colab.research.google.com/github/microsoft/autogen/blob/main/notebook/agentchat_web_info.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AutoGen Agents with Retrieval Augmented Generation

**`AutoGen`** is a versatile framework that facilitates the creation of LLM applications by employing multiple agents capable of interacting with one another to tackle tasks. These AutoGen agents can be tailored to specific needs, engage in conversations, and seamlessly integrate human participation. They are adaptable to different operation modes that encompass the utilization of LLMs, human inputs, and various tools.

**`RAG`** stands for `Retrieval Augmented Generation`, a natural language processing (NLP) technique that combines two essential components: **retrieval** and **generation**.

The previous tutorial [AutoGen + LangChain = Super AI Agents](https://github.com/sugarforever/LangChain-Advanced/blob/main/Integrations/AutoGen/autogen_langchain_uniswap_ai_agent.ipynb) introduced how to build an AutoGen application that can execute tasks requiring specific documents knowledge. This is a typical RAG use case, aka. document based chatbot.

The latest **AutoGen** version already supports RAG natively with the feature package `retrievechat`.

In this tutorial, we are going to rebuild the same feature demonstrated in the previous tutorial. We will utilize `AutoGen` `retrievechat` feature.

This tutorial is inspired by the [Blog - Retrieval-Augmented Generation (RAG) Applications with AutoGen](https://microsoft.github.io/autogen/blog/2023/10/18/RetrieveChat) of [Li Jiang](https://github.com/thinkall).

Credits go to Li Jiang! 🙌

Let's roll! 🚴🏻‍♀️ 🚴🏻 🚴🏻‍♂️

## Use Case



In this tutorial, I will create the retrieval augmented agents with the following document:

[RETRIEVAL AUGMENTED GENERATION AND REPRESENTATIVE
VECTOR SUMMARIZATION FOR LARGE UNSTRUCTURED
TEXTUAL DATA IN MEDICAL EDUCATION](https://arxiv.org/pdf/2308.00479.pdf)

You should be able to see the agents are able to perform retrieval augmented generation based on the document above and answer question.

### Environment Preparation

In [ ]:
%pip install pyautogen[retrievechat] langchain "chromadb<0.4.15" -q

In [ ]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4"],
    },
)

### Steps

#### 1. Configure Embedding Function

We will use OpenAI embedding function.

In [ ]:
from chromadb.utils import embedding_functions

openai_embedding_function = embedding_functions.OpenAIEmbeddingFunction(api_key = config_list[0]["api_key"])

#### 2. Configure Text Splitter

LangChain has done a great job in text splitting, so we will use its components.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n", "\r", "\t"])

#### 3. Configure Vector Store

By default, the AutoGen retrieval augmented agents use `chromadb` as vector store.

Developers can configure preferred vector store by extending the class `RetrieveUserProxyAgent` and overriding function `retrieve_docs`.

AutoGen also supports simple configuration items to customize Chromadb storage.

In this demo, we will specify the collection name by `retreive_config` item `collection_name`. You should be able to see it in step 4.


#### 4. Create Retrieval Augmented Agents

In [ ]:
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

llm_config = {
    "request_timeout": 600,
    "config_list": config_list,
    "temperature": 0
}

assistant = RetrieveAssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    llm_config=llm_config,
)

rag_agent = RetrieveUserProxyAgent(
    human_input_mode="NEVER",
    retrieve_config={
        "task": "qa",
        "docs_path": "./rag.pdf",
        "collection_name": "rag_collection",
        "embedding_function": openai_embedding_function,
        "custom_text_split_function": text_splitter.split_text,
    },
)



### It's time to start a chat with the RAG agent.

In [ ]:
assistant.reset()
rag_agent.initiate_chat(assistant, problem="What is the workflow in docGPT?", n_results=2)

INFO:autogen.retrieve_utils:Found 3 chunks.


Trying to create collection.
doc_ids:  [['doc_1', 'doc_2']]
Adding doc_id doc_1 to context.
Adding doc_id doc_2 to context.
RetrieveChatAgent (to assistant):

You're a retrieve augmented chatbot. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
You must give as short an answer as possible.

User's question is: What is the workflow in docGPT?

Context is: is initially stored in memory as a Document and then the Document is split into text chunks using recursive character text
splitting. The chunks are then embedded in a 1536-dimensional vector space using OpenAI text-embedding-ada-002
embedding engine and stored in Facebook AI Similarity Search ( FAISS ) vector database [ 3]. The FAISS vector database
is used to find the k-most similar chunks to a given query at the query time. The original query, combined with the
retrieved chunks

## Autogen Flowise LangChain together RAG bot

Flowise作为低代码无代码的AI流创建工具，迅速构建RAG的AI 流程，然后暴露出API，提供给Autogen作为Tool使用

Flowise生成的接口如下：

``` python
import requests

API_URL = "http://localhost:4000/api/v1/prediction/433ed37e-9546-4e73-a688-7352b78bf852"

def query(payload):
    response = requests.post(API_URL, json=payload)
    return response.json()

output = query({
    "question": "Hey, how are you?",
})
```

通过自定义函数，提供给Autogen框架使用

``` python
import requests

API_URL = "http://localhost:4000/api/v1/prediction/433ed37e-9546-4e73-a688-7352b78bf852"

def answer_flowise_question(question):
  response = requests.post(API_URL, json={ "question": question })
  return response.json()
```

设置AutoGen UserProxyAgent和AssistantAgent: 

``` python
llm_config={
    "config_list": config_list,
    "temperature": 0,
    "functions": [
        {
            "name": "answer_flowise_question",
            "description": "Answer any related questions",
            "parameters": {
                "type": "object",
                "properties": {
                    "question": {
                        "type": "string",
                        "description": "The question to ask in relation to my document",
                    }
                },
                "required": ["question"],
            },
        }
    ],
}

assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
)

user_proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=10,
    code_execution_config={"work_dir": "."},
    llm_config=llm_config,
    system_message="""Reply TERMINATE if the task has been solved at full satisfaction.
Otherwise, reply CONTINUE, or the reason why the task is not solved yet.""",
    function_map={"answer_flowise_question": answer_flowise_question}
)
```


现在就可以用user_proxy提交任务了。代码如下：

``` python
user_proxy.initiate_chat(
    assistant,
    message="""
I'm writing a blog to introduce the version 3 of Uniswap protocol. Find the answers to the 3 questions below and write an introduction based on them.

1. What is Uniswap?
2. What are the main changes in Uniswap version 3?
3. How to use Uniswap?

Start the work now.
"""
)
```